# conductive.ai Take-home assignment

## Description:
A semiconductor fab is performing a SATEOS CVD process on 4 different AMAT PRODUCER SE tools.

They have performed Design of Experiments on 500 300mm wafers by varying 3 process parameters:

- Flowfactor: inversely proportional to TEOS (precursor) flow. Actual flow value not accessible at the moment.
- Spacing: distance between shower head and wafer.
- Deposition Time.

Temperature was held constant at 480 celsius. The fab wants to know how would film thickness change if they would change the temperature in the range of 460-500 celsius.

They chose 125 parameter combinations (recipes) and performed each of them on 4 different tools to account for tool variation.

Film thickness values were measured (in angstrom) at 49 specific points on the wafer (sites). The spec is 1500Å.

Coordinates of the sites are given in a separate file. Values are in micrometers, (0,0) is the center of the wafer.

### Task:
 - Reconstruct wafer profile images from sites using interpolation.
 - Build a generative machine learning model (VAE, GAN, or similar) that predicts the thickness profile of the wafer based on flowfactor, spacing, deposition time, and tool.
 - Ensure that the machine learning model is capable of extrapolating 1 standard deviation outside of the ranges of parameters in accordance with physics.

- (Bonus) Build a physics-based model (preferably using simulations) for this process that would include temperature. If you need information on other process parameters, we'd be happy to provide them if available. Adapt it to fit the data. Combine it with the ML model to incorporate the influence of temperature.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, LeakyReLU, ReLU, Flatten, Dropout, Dense, Reshape, Conv2DTranspose, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.layers import Input, Embedding, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import Mean
import tensorflow as tf
# from tensorflow import InteractiveSession
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.pyplot import figure, imshow, axis
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas

from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

import pandas as pd
import numpy as np


In [ ]:
%pylab inline

Reading the data:

In [ ]:
df = pd.read_csv('aug_df.csv',)
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
df

In [ ]:
features = ['FLOWFACTOR', 'SPACING', 'DEP TIME', 'TOOL']
all_sites = df.columns.drop(features)

### Defining weights of the samples:

In [ ]:
Y_COLUMNS = ['FLOWFACTOR', 'SPACING', 'DEP TIME']
scaler_y = MinMaxScaler()

y = df[Y_COLUMNS]
y_scaled = scaler_y.fit_transform(y)

tsne=TSNE()
y_tsne = tsne.fit_transform(y_scaled)

In [ ]:
#number of bins per each extrinsic properties
BINS=6

for Y_column in Y_COLUMNS:
    print(Y_column)
    fig, axs= plt.subplots(1,2, figsize=(7,3))
    im=axs[0].scatter(y_tsne[:,0], y_tsne[:,1], c= df[Y_column]);
    h=axs[1].hist(df[Y_column], bins=BINS)
    plt.show()

binning_labels_dict={}
binning_labels_dict['TOOL_binning'] = df['TOOL'].values

Y_COLUMNS_BINNING=['TOOL_binning']
for Y_column in Y_COLUMNS:
    cnt, bins = np.histogram(df[Y_column], bins=BINS)
    bins[0] -= 1
    col_name = Y_column + "_binning"
    binning_labels_dict[col_name] = np.searchsorted(bins, df[Y_column].values)
    Y_COLUMNS_BINNING.append(col_name)
    

In [ ]:
binning_df = pd.DataFrame(binning_labels_dict)#.groupby(col_name).count()
binning_df["T"] = 1
count_bins_df = binning_df.groupby(Y_COLUMNS_BINNING).count()
max_count = count_bins_df["T"].max()

df = pd.concat((df,binning_df),axis=1)
df = df.drop("T",axis=1)
count_bins_df = count_bins_df.reset_index()
df = pd.merge(df,count_bins_df,on=Y_COLUMNS_BINNING)
df["weight"] = max_count/df["T"]

## One-hot encoding of TOOL feature:

In [ ]:
df = pd.get_dummies(df, columns=['TOOL'] )

In [ ]:
df

In [ ]:
Y_COLUMNS = ['FLOWFACTOR', 'SPACING', 'DEP TIME', 'TOOL_1', 'TOOL_2', 'TOOL_3', 'TOOL_4']

In [ ]:
df

## Data preproccesing and rescaling for the model

The thickness profile array with the shape of (49,) reshaped to a 7x7 matrix for the convolutional networks

In [ ]:
BATCH_SIZE=16
batch_count = int(np.ceil(len(df)/BATCH_SIZE))

WEIGHTS=[]

# reshape the thickness array to 7x7 matrix and rescale
scaler_x = MinMaxScaler((-1,1))
XS_scaled = scaler_x.fit_transform(df[all_sites])
XS_scaled = XS_scaled.reshape(15500, 7, 7)

scaler_y_2 = MinMaxScaler((-1,1))
YS = np.array(df[Y_COLUMNS]) #for testing the model only
YS_scaled = scaler_y_2.fit_transform(df[Y_COLUMNS])

for b in range(0, batch_count):
#     print(b * BATCH_SIZE,"->",min(len(df), b * BATCH_SIZE+BATCH_SIZE) - 1)
    batch_indices = np.arange(b * BATCH_SIZE, min(len(df), b * BATCH_SIZE + BATCH_SIZE))
    for ind in batch_indices:              
         WEIGHTS.append(df.loc[ind,"weight"])
        

WEIGHTS = np.array(WEIGHTS)
dataset=(XS_scaled, YS_scaled, WEIGHTS)

## Discriminator Model

In [ ]:
INPUT_SHAPE=(7, 7, 1)
EXTRINSIC_DIM = 7 #LABELS, Y
latent_dim = 32 # need to confirm
LATENT_DIM = latent_dim

In [ ]:
# define the standalone discriminator model
# @tf.function
def define_cond_discriminator(in_shape=INPUT_SHAPE, n_classes=EXTRINSIC_DIM):    
    # label input
    in_label = Input(shape=(EXTRINSIC_DIM,), name="Y_extrinsic")
    # scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1]

    li = Dense(n_nodes)(in_label)

    # reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], 1))(li)

    # image input
    in_image = Input(shape=in_shape, name="X")
    # concat label as a channel
    merge = Concatenate()([in_image, li])
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
#     fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
#     fe = Dropout(0.3)(fe)
    # downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
#     fe = BatchNormalization()(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # dropout
    fe = Dropout(0.4)(fe)
    # output
    out_layer = Dense(1, activation='sigmoid',name="Discriminator")(fe)
    # define model
    model = Model([in_image, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.00001, beta_1=0.5)
    #opt = Adadelta()

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
discriminator_model = define_cond_discriminator()

In [ ]:
discriminator_model.summary()

## Generator Model

In [ ]:

# define the standalone generator model
def define_cond_generator(latent_dim=LATENT_DIM, n_classes=EXTRINSIC_DIM):
    init_size=(7, 7)

    # label input
    in_label = Input(shape=(EXTRINSIC_DIM,), name="Y_extrinsic")
    # linear multiplication
    n_nodes = init_size[0] * init_size[1]
    li = Dense(n_nodes)(in_label)
#     li = BatchNormalization()(li)
#     li = Dropout(0.3)(li)

    # reshape to additional channel
    li = Reshape((init_size[0] , init_size[1], 1))(li)

    # image generator input
    in_lat = Input(shape=(latent_dim,), name="Z_latent")
    # foundation for 7x7 image
    n_nodes = 128 * n_nodes
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = BatchNormalization()(gen)
    gen = Dropout(0.5)(gen)
    
    gen = Reshape((init_size[0], init_size[1], 128))(gen)

    # merge image gen and label input
    merge = Concatenate()([gen, li])
   
#     gen = Conv2DTranspose(128, (3,3), strides=(1,1), padding='same')(merge)
#     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = BatchNormalization()(gen)
#     gen = Dropout(0.2)(gen)

#     gen = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same')(gen)
#     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = BatchNormalization()(gen)
#     gen = Dropout(0.2)(gen)
    
#     gen = Conv2D(128, (3,3), strides=(2,2), padding='same')(gen)
#     gen = BatchNormalization()(gen)
#     gen = LeakyReLU(alpha=0.2)(gen)
# #     fe = Dropout(0.3)(fe)
#     # downsample
#     gen = Conv2D(128, (3,3), strides=(2,2), padding='same')(gen)
#     gen = LeakyReLU(alpha=0.2)(gen)
#     gen = BatchNormalization()(gen)
    
#     gen = Reshape((5, 10, 1))(gen)

    # output
    out_layer = Conv2D(1, (7, 7),activation='tanh', padding='same', name="X_generated")(gen)

    # define model
    model = Model([in_lat, in_label], out_layer)
    return model

In [ ]:
generator_model = define_cond_generator()

    

In [ ]:
generator_model.summary()

## Combine generator and discriminator model

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_cond_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # define gan model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
GAN = define_cond_gan(generator_model, discriminator_model)

# Generate Real and Fake Samples:

In [ ]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into vector and labels
    vector, labels, weights = dataset
    # choose random instances
    ix = np.random.randint(0, vector.shape[0]-1, n_samples) # n_samples were removed I dunno why
    # select vector and labels
    X, labels, weights = vector[ix], labels[ix], weights[ix]
    # generate class labels
    y = np.ones((n_samples, 1))
    return [X, labels], y, weights

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=EXTRINSIC_DIM):
    # generate points in the latent space
    x_input = np.random.randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = np.random.choice(len(YS_scaled), size=n_samples)
    weights = WEIGHTS[labels]
    labels= YS_scaled[labels]    
    return z_input, labels, weights

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input, weights_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = np.zeros((n_samples, 1))
    return [images, labels_input], y, weights_input

## Trainig the GAN model function:

In [ ]:
# function for trainining of generator and discriminator
def train_cgan(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=32, verbose_freq=10, n_batch=32):
    fig,ax = plt.subplots(1,1)
    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)

    # manually enumerate epochs
    d_loss1_epochs=[]
    d_loss2_epochs=[]
    g_loss_epochs=[]
    d_loss1_batch=[]
    d_loss2_batch=[]
    g_loss_batch=[]
    
    
    # manually enumerate epochs
    iteration=0
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            [X_real, labels_real], y_real, weights_real = generate_real_samples(dataset, half_batch)
            # update discriminator model weights
            d_loss1, _ = d_model.train_on_batch([X_real, labels_real], y_real,) # sample_weight=weights_real)
            # generate 'fake' examples
            [X_fake, labels], y_fake, weights_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update discriminator model weights
            d_loss2, _ = d_model.train_on_batch([X_fake, labels], y_fake,)#sample_weight=weights_fake)
            # prepare points in latent space as input for the generator
            z_input, labels_input, weights_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = np.ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan,) #sample_weight=weights_gan)
            
            d_loss1_batch.append(d_loss1)
            d_loss2_batch.append(d_loss2)
            g_loss_batch.append(g_loss)
        

            # visualizing the training steps
            if (iteration%verbose_freq==0):
                if ax.lines:
                    epochs_x = (np.arange(len(d_loss2_batch))+1)/bat_per_epo
                    for line,ydata in zip(ax.lines,[d_loss1_batch, d_loss2_batch,g_loss_batch]):
                        line.set_xdata(epochs_x)
                        line.set_ydata(ydata) 
                    ax.set_xlim(0,max(epochs_x) + 1/bat_per_epo)
                    ax.set_ylim(0.9 * min([min(d_loss1_batch), min(d_loss2_batch), min(g_loss_batch)]), 1.1 * max([max(d_loss1_batch), max(d_loss2_batch), max(g_loss_batch)]))
                else:
                    ax.plot(d_loss1_batch, label="d_loss_real")
                    ax.plot(d_loss2_batch, label="d_loss_fake")
                    ax.plot(g_loss_batch, label="g_loss")                 
                    fig.legend()
                fig.canvas.draw()
                
            iteration += 1
            
    if ax.lines:        
        epochs_x=(np.arange(len(d_loss2_batch))+1)/bat_per_epo
        for line,ydata in zip(ax.lines,[d_loss1_batch, d_loss2_batch,g_loss_batch]):
            line.set_xdata(epochs_x)
            line.set_ydata(ydata) 
        ax.set_xlim(0,max(epochs_x)+1/bat_per_epo)
        ax.set_ylim(0.9*min([min(d_loss1_batch),min(d_loss2_batch),min(g_loss_batch)]),1.1*max([max(d_loss1_batch), max(d_loss2_batch), max(g_loss_batch)]))
    else:
        ax.plot(d_loss1_batch, label="d_loss_real")
        ax.plot(d_loss2_batch, label="d_loss_fake")
        ax.plot(g_loss_batch, label="g_loss")                 
        fig.legend()
    fig.canvas.draw()

### Process vector from the inputs and visualization functions:

In [ ]:
def create_process_vector(features):
    features = np.array(features).reshape(1, -1)
    process_vector = scaler_y_2.transform(features)
    return process_vector

In [ ]:
import plotly.figure_factory as ff
from scipy.spatial import Delaunay

def visualize(thickness):
    site = pd.read_csv('site_coordinates.csv')
    site['SITE_Z'] = thickness
    points2D = np.vstack([site['SITE_X'], site['SITE_Y']]).T
    tri = Delaunay(points2D)
    simplices = tri.simplices

    fig = ff.create_trisurf(site['SITE_X'], site['SITE_Y'], site['SITE_Z'],
                             simplices=simplices,
                             title="wafare", aspectratio=dict(x=1, y=1, z=0.5))
    fig.show()


In [ ]:
def visualize_sample(i):
    pp = create_process_vector(YS[i])
    latent_points, labels, _ = generate_latent_points(latent_dim, 1)
    del labels
    X_test = generator_model.predict([latent_points, pp])
    X_test = scaler_x.inverse_transform(X_test.reshape(1,-1))
    visualize(X_test[0])

### Training:

In [ ]:
%matplotlib notebook
# train model
n_runs = 64
total_batch = 0
total_epochs = 0
n_batch = 128
n_epochs = 16

print('train model for ' + str(n_runs * n_epochs))
for k in range(n_runs):
    print(k)
    total_batch = total_batch + (dataset[0].shape[0] / n_batch) * n_epochs
    print(total_batch)
    train_cgan(generator_model, discriminator_model, GAN, dataset, latent_dim)
    
    #visualize samples after each run
    visualize_sample(0)
    visualize_sample(1)
    visualize_sample(3)
    visualize_sample(4)
    

    # save models
    savename = '7x7_aug_data_5'
    generator_model.save(savename+'_total_batches_'+str(total_batch) + '_generator.h5')
    discriminator_model.save(savename+'_total_batches_'+str(total_batch) + ' _discriminator.h5')
    GAN.save(savename+'_total_batches_'+str(total_batch) + '_gan.h5')

In [ ]:
from tensorflow.keras.models import model_from_json

def save_model(model):
    # serialize model to JSON
    with open(f"{model}.json", "w") as json_file:
        json_file.write(generator_model.to_json())

    # serialize weights to HDF5
    generator_model.save_weights(f"{model}.h5")
    print("Saved model to disk")
    
save_model('generator_model')

In [ ]:
# latent_points, labels, _ = generate_latent_points(latent_dim, 1)
visualize_sample(0)
# visualize_sample(1)
# visualize_sample(3)
# visualize_sample(4)
